In [43]:
!pip install pycryptodome
!pip install cryptography

In [64]:
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.PublicKey import RSA
from Crypto.Signature import pkcs1_15
from Crypto.Hash import SHA1, SHA256, MD5
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad
import hashlib
import hmac

# AES Encryption/Decryption

def aes_key(key_size):
    key = get_random_bytes(key_size)
    return key

def aes_encrypt_decrypt(mode, operation, data, key, iv=None):
    if mode == 'ECB' and operation == 'encrypt':
        cipher = AES.new(key, AES.MODE_ECB)
        padded_data = pad(data, AES.block_size)
        return cipher.encrypt(padded_data)
    elif mode == 'ECB' and operation == 'decrypt':
        cipher = AES.new(key, AES.MODE_ECB)
        try:
            data= data.decode()
            decrypted_data = cipher.decrypt(data)
            unpadded_data = unpad(decrypted_data, AES.block_size)
            return unpadded_data
        except ValueError:
            raise ValueError("Padding is incorrect. Decryption failed.")
    elif mode == 'CBC' and operation == 'encrypt':
        if iv is None:
            iv = get_random_bytes(AES.block_size)
        cipher = AES.new(key, AES.MODE_CBC, iv=iv)
        padded_data = pad(data, AES.block_size)
        ciphertext = cipher.encrypt(padded_data)
        return iv + ciphertext
    elif mode == 'CBC' and operation == 'decrypt':
        iv = data[:AES.block_size]
        cipher = AES.new(key, AES.MODE_CBC, iv=iv)
        try:
            decrypted_data = cipher.decrypt(data[AES.block_size:])
            return unpad(decrypted_data, AES.block_size)
        except ValueError:
            raise ValueError("Padding is incorrect. Decryption failed.")
    else:
        raise ValueError("Invalid mode or operation")

# RSA Encryption/Decryption
def rsa_encrypt_decrypt(operation, data, key):
    if operation == 'encrypt':
        cipher = PKCS1_OAEP.new(key)
        return cipher.encrypt(data)
    elif operation == 'decrypt':
        x= RSA.decrypt(data, key).decode()
        # cipher = PKCS1_OAEP.new(key)
        # return cipher.decrypt(data)
        return x
    else:
        raise ValueError("Invalid operation")

# Hashing
def hash_data(data, hash_mode):
    if hash_mode == 'SHA1':
        h = SHA1.new(data)
    elif hash_mode == 'SHA256':
        h = SHA256.new(data)
    elif hash_mode == 'MD5':
        h = hashlib.md5(data)
    else:
        return
    return h.hexdigest()

# Digital Signature using RSA
def generate_rsa_key_pair():
    key = RSA.generate(2048)
    private_key = key.export_key()
    public_key = key.publickey().export_key()
    return private_key, public_key

private_key, public_key = generate_rsa_key_pair()


def rsa_sign_verify(operation, message, signature, private_key, public_key):
    if operation in  ('generation', '1'):
        private_key= RSA.import_key(private_key)
        signature = pkcs1_15.new(private_key).sign(SHA256.new(message))
        return signature
    elif operation in ('verification', '2'):
        public_key= RSA.import_key(public_key)
        verifier= pkcs1_15.new(public_key)
        # print(SHA256.new(message))

        try:
            verifier.verify(SHA256.new(message), signature)
            return True
        except (ValueError, TypeError):
            return False

# MAC Generation
def generate_key():
  mac_key = get_random_bytes(32)
  return mac_key

mac_key= generate_key()

def generate_mac(data, mac_key, mac_algorithm):
    try:
        mac = hmac.new(mac_key, data, hashlib.__dict__[mac_algorithm])
        return mac.hexdigest()
    except KeyError:
        return None


def main(session):

    while True:
        print("1. AES Encryption/Decryption")
        print("2. RSA Encryption/Decryption")
        print("3. Hashing")
        print("4. Digital Signature using RSA")
        print("5. MAC Generation")
        print("6. Exit")


        choice = input("Enter your choice: ")


        if choice == '1':
            mode = input("Enter mode (ECB/CBC): ").upper()
            operation = input("Enter operation (Encrypt/Decrypt): ").lower()
            data = input("Enter plaintext/ciphertext: ").encode()
            # key=''
            if session:
              key_size = int(input("Enter key size (16/24/32 bytes): "))
              if key_size in (16, 24, 32):
                  key= aes_key(key_size)
                  print("key: " , key)
                  session= False
              else:
                  print("\n ----------Wrong Key Size!--------- \n")
                  return

            # key = input("Enter key (16/24/32 bytes): ").encode()
            result = aes_encrypt_decrypt(mode, operation, data, key)
            print("\n" + "Result:", result)
            print('\n')

        elif choice == '2':
            operation = input("Enter operation (Encrypt/Decrypt): ").lower()
            data = input("Enter plaintext/ciphertext: ").encode()
            key = RSA.generate(2048)
            result = rsa_encrypt_decrypt(operation, data, key)
            print("\n" + "Result:", result)
            print('\n')

        elif choice == '3':
            plaintext = input("Enter plaintext: ").encode()
            hash_mode = input('Enter hash mode: ').upper()
            while True:
              if hash_mode in ['SHA1', 'SHA256', 'MD5']:
                result = hash_data(plaintext,hash_mode)
                print("\n" + "Hash:", result + '\n')
              else:
                print("\n" + "Invalid hash mode! Returning to the main menu. \n")
              break


        elif choice == '4':
            operation = input("Enter operation (1: Generation / 2: Verification): ").lower()
            message = input("Enter message: ").encode('utf-8')
            # private_key = RSA.generate(2048)
            # public_key = private_key.publickey()
            # key = RSA.generate(2048)
            # private_key = key.export_key()
            # public_key = key.publickey().export_key()
            signature=''
            if operation in ('generation', '1'):
                signature = rsa_sign_verify(operation, message, signature, private_key, public_key)
                print("\n" + "Signature:", signature.hex() + '\n')
            elif operation in  ('verification', '2'):
                signature = bytes.fromhex(input("Enter signature: "))
                # print(signature)
                # decoded_signature = binascii.b2a_hex(signature).decode()
                # print(decoded_signature)
                result = rsa_sign_verify(operation, message, signature, private_key, public_key)
                if result:
                    print("\n" + "Signature is valid. \n ")
                else:
                    print("\n" + "Signature is not valid. \n")

        elif choice == '5':
            message = input("Enter message: ").encode()
            mac_algorithm = input("Select the MAC algorithm (sha256/sha512): ").lower()
            mac = generate_mac(message, mac_key, mac_algorithm)
            print('\n' + "Key:", mac_key.hex())
            print("\n" + "MAC:", mac + '\n')


        elif choice =='6':
          break

        else:
            print("Invalid choice. \n")

if __name__ == "__main__":
  session=True
  main(session)

1. AES Encryption/Decryption
2. RSA Encryption/Decryption
3. Hashing
4. Digital Signature using RSA
5. MAC Generation
6. Exit
Enter your choice: 1
Enter mode (ECB/CBC): ecb
Enter operation (Encrypt/Decrypt): encrypt
Enter plaintext/ciphertext: cryptography
Enter key size (16/24/32 bytes): 24
key:  b"\xafK:\x8aZ\x19\xd1\x1b'\x19\xb7q\xd4\x9d6\x05\x95$;\xcd\xec\x1f\x89#"

Result: b'\x8d\xbe\x9d\x86\x9d\x15L\xf1\xde\xbe\xbbZ\rY1\r'


1. AES Encryption/Decryption
2. RSA Encryption/Decryption
3. Hashing
4. Digital Signature using RSA
5. MAC Generation
6. Exit
Enter your choice: 11
Invalid choice. 

1. AES Encryption/Decryption
2. RSA Encryption/Decryption
3. Hashing
4. Digital Signature using RSA
5. MAC Generation
6. Exit
Enter your choice: 1
Enter mode (ECB/CBC): cbc
Enter operation (Encrypt/Decrypt): encrypt
Enter plaintext/ciphertext: cryptoanalysis

Result: b'H\x04\xa1\x11\xe2\xc0I\xde\x10\xba\xb7\xc6l0(=\x9e\xda\xee~\xe3o:\x1c\xd3\x92\x8f\rja\xbfr'


1. AES Encryption/Decryption
2. RSA

KeyboardInterrupt: ignored

In [57]:
ciphertext =b'\x91\xc9|e\xa7\xc5\xc0c\xebG\x8f\x8d\x05\x1e\x89\xe1l\x87\xdew\xb5\xb9\xea\xa99b\x06@A\xf29k'
#  b'\x91\xc9|e\xa7\xc5\xc0c\xebG\x8f\x8d\x05\x1e\x89\xe1l\x87\xdew\xb5\xb9\xea\xa99b\x06@A\xf29k'
import base64
ciphertext_base64 = base64.b64encode(ciphertext).decode('utf-8')
print(ciphertext_base64)
ciphertext_hex = ciphertext.hex()
print(ciphertext_hex)

# x= "i am nothing"

# ciphert= RSA.encrypt(x.encode(), b'\x91\xc9|e\xa7\xc5\xc0c\xebG\x8f\x8d\x05\x1e\x89\xe1l\x87\xdew\xb5\xb9\xea\xa99b\x06@A\xf29k' )
# print(x)

kcl8ZafFwGPrR4+NBR6J4WyH3ne1ueqpOWIGQEHyOWs=
91c97c65a7c5c063eb478f8d051e89e16c87de77b5b9eaa93962064041f2396b


In [61]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes
import base64
key= b'*e\x80\xd8$>~?\xafU\x94\x00/\xce+\x90<T\x8b\x0bK\xe3\xd1G\x1f\x040\xf3\xb7v\x94E'
# data= "abcd".encode()
data= b'\x91\xc9|e\xa7\xc5\xc0c\xebG\x8f\x8d\x05\x1e\x89\xe1l\x87\xdew\xb5\xb9\xea\xa99b\x06@A\xf29k'.decode()
cipher = AES.new(key, AES.MODE_ECB)
padded_data = pad(data, AES.block_size)
print(cipher.encrypt(padded_data))

decrypted_data = cipher.decrypt(data)
print(unpad(decrypted_data, AES.block_size))

# x= x.encrypt(x)
# print(x)

UnicodeDecodeError: ignored